<table style="width: 100%;">
    <tr>
        <td><a href="https://ieb-chile.cl/en/" target="_blank"><img src="https://raw.githubusercontent.com/IEB-BIODATA/pydwca-examples/main/images/logo/IEB.png" style="height: 100px;"></a</td>
        <td></td>
        <td><img src="https://raw.githubusercontent.com/IEB-BIODATA/pydwca-examples/main/images/logo/Biodata.png" style="height: 100px;"></td>
    </tr>
</table>

# Catalogue Of Life

A usage example of the `PyDwCA` library, reading the DwC-A of Catalogue of Life.

Reading the [Catalogue of Life](https://www.catalogueoflife.org/data/download):

In [1]:
import os
import pandas as pd
os.makedirs("data", exist_ok=True)

In [2]:
!wget https://download.catalogueoflife.org/col/annual/2023_dwca.zip -O data/col-dwca.zip

--2024-08-10 16:58:16--  https://download.catalogueoflife.org/col/annual/2023_dwca.zip
Resolving download.catalogueoflife.org (download.catalogueoflife.org)... 130.225.43.13
Connecting to download.catalogueoflife.org (download.catalogueoflife.org)|130.225.43.13|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://download.checklistbank.org/col/annual/2023_dwca.zip [following]
--2024-08-10 16:58:17--  https://download.checklistbank.org/col/annual/2023_dwca.zip
Resolving download.checklistbank.org (download.checklistbank.org)... 130.225.43.13
Connecting to download.checklistbank.org (download.checklistbank.org)|130.225.43.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 398371642 (380M) [application/zip]
Saving to: ‘data/col-dwca.zip’

data/col-dwca.zip   100%[===================>] 379.92M  14.2MB/s    in 29s     

2024-08-10 16:58:46 (13.1 MB/s) - ‘data/col-dwca.zip’ saved [398371642/398371642]



In [3]:
from dwca import DarwinCoreArchive

In [4]:
col = DarwinCoreArchive.from_file("data/col-dwca.zip")

/home/ubuntu/pydwca/src/dwca/classes/data_file.py:198: UserWarning: https://terms.catalogueoflife.org/notho not in expected namespace for http://rs.tdwg.org/dwc/terms/Taxon class. Some functionalities may not be available.
  warn(f"{element.get('term')} not in expected namespace for "
/home/ubuntu/pydwca/src/dwca/base/darwincore_archive.py:76: UserWarning: http://rs.gbif.org/terms/1.0/Distribution not in expected namespace. Some functionalities may not be available.
  warn(f"{element.get('rowType')} not in expected namespace. "
/home/ubuntu/pydwca/src/dwca/base/darwincore_archive.py:76: UserWarning: http://rs.gbif.org/terms/1.0/SpeciesProfile not in expected namespace. Some functionalities may not be available.
  warn(f"{element.get('rowType')} not in expected namespace. "
/home/ubuntu/pydwca/src/dwca/base/darwincore_archive.py:76: UserWarning: http://rs.gbif.org/terms/1.0/VernacularName not in expected namespace. Some functionalities may not be available.
  warn(f"{element.get('rowTyp

In [5]:
print(col)

col-clb-9910 [Core: http://rs.tdwg.org/dwc/terms/Taxon, Entries: 4983455]


In [6]:
taxon = col.core
taxon.pandas

Converting to pandas: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4983455/4983455 [00:02<00:00, 1869204.18entry/s]


,taxonID,parentNameUsageID,acceptedNameUsageID,originalNameUsageID,scientificNameID,datasetID,taxonomicStatus,taxonRank,scientificName,scientificNameAuthorship,...,infragenericEpithet,specificEpithet,infraspecificEpithet,cultivarEpithet,nameAccordingTo,namePublishedIn,nomenclaturalCode,nomenclaturalStatus,taxonRemarks,references
0,5T6MX,,,,0,,accepted,unranked,Biota,,...,,,,,,,,nomen legitimum,,
1,5SY,HP,,,103a9c28-0651-4695-959f-967f5e2c658c,,accepted,superfamily,Velocipedoidea,,...,,,,,,,,,,
2,3FD8,624LX,,,e1ae1313-a1f2-4fec-ace6-bba4a245a27c,1171,accepted,genus,Callichilella,,...,,,,,,,,,,
3,63DJ6,624LX,,,1c742263-db72-46e5-aa8c-c6f56aa64a35,1171,accepted,genus,Neoloxops,,...,,,,,,,,,,
4,5Q35,624LX,,,e7eeed09-0d71-4cd7-8c62-a752ff953f09,1171,accepted,genus,Mesotropiscus,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4983450,4NMGW,,5KNTL,,430513f4-35ef-4068-8fee-b4149f1defda,1037,synonym,species,"Pseudepipona lateralis (Fabricius, 1781)","(Fabricius, 1781)",...,,lateralis,,,,,ICZN,,,
4983451,B8W3X,,5KNTM,,5254b6b3-04c9-4981-9252-a3788db3ba3e,1037,ambiguous synonym,species,"Proepipona laterale (Fabricius, 1781)","(Fabricius, 1781)",...,,laterale,,,,,ICZN,,,
4983452,B9M49,,5KNTM,,e2aa97c9-c677-43bd-9b70-0b0089e0840c,1037,ambiguous synonym,species,"Odynerus lateralis (Fabricius, 1781)","(Fabricius, 1781)",...,,lateralis,,,,,ICZN,,,
4983453,4PVZ8,,5KQFS,,54767f71-35d3-4dbf-a576-9ee48c1fbbfe,1037,synonym,species,"Pterochilus glabripalpis de Saussure, 1856","de Saussure, 1856",...,,glabripalpis,,,,,ICZN,,,


## Exploring this `DataFrame`

In [7]:
taxon.pandas.groupby("taxonRank").size()

taxonRank
aberration                   338
class                        455
epifamily                      6
family                     14420
form                       56745
genus                     248116
gigaclass                      2
infraclass                    24
infrageneric name            584
infraorder                   175
infraphylum                    2
infraspecific name         25182
infrasubspecific name         90
infratribe                    43
kingdom                       18
megaclass                      1
morph                          4
mutatio                       25
nanorder                      11
natio                          4
order                       1748
other                       1179
parvorder                     29
parvphylum                     2
phylum                       132
proles                       630
realm                          6
section                      275
species                  4045739
species aggregate              1


In [8]:
taxon.pandas.groupby(["taxonRank", "taxonomicStatus"]).size().unstack(fill_value=0)

taxonomicStatus,accepted,ambiguous synonym,misapplied,provisionally accepted,synonym
taxonRank,,,,,
aberration,0,14,0,0,324
class,411,0,0,7,37
epifamily,6,0,0,0,0
family,13322,0,0,188,910
form,824,923,1,6,54991
genus,203435,43,5,3086,41547
gigaclass,2,0,0,0,0
infraclass,21,0,0,3,0
infrageneric name,0,0,0,0,584


Because we use the `"acceptedNameUsageID"` to find synonyms, we are going to fill this columns for all the `"accepted"` taxa with its own `"taxonID"`:

In [9]:
accepted_mask = taxon.pandas["taxonomicStatus"].str.contains("accepted")
taxon.pandas.loc[
    taxon.pandas[accepted_mask].index, "acceptedNameUsageID"
] = taxon.pandas[accepted_mask]["taxonID"]
taxon.pandas

,taxonID,parentNameUsageID,acceptedNameUsageID,originalNameUsageID,scientificNameID,datasetID,taxonomicStatus,taxonRank,scientificName,scientificNameAuthorship,...,infragenericEpithet,specificEpithet,infraspecificEpithet,cultivarEpithet,nameAccordingTo,namePublishedIn,nomenclaturalCode,nomenclaturalStatus,taxonRemarks,references
0,5T6MX,,5T6MX,,0,,accepted,unranked,Biota,,...,,,,,,,,nomen legitimum,,
1,5SY,HP,5SY,,103a9c28-0651-4695-959f-967f5e2c658c,,accepted,superfamily,Velocipedoidea,,...,,,,,,,,,,
2,3FD8,624LX,3FD8,,e1ae1313-a1f2-4fec-ace6-bba4a245a27c,1171,accepted,genus,Callichilella,,...,,,,,,,,,,
3,63DJ6,624LX,63DJ6,,1c742263-db72-46e5-aa8c-c6f56aa64a35,1171,accepted,genus,Neoloxops,,...,,,,,,,,,,
4,5Q35,624LX,5Q35,,e7eeed09-0d71-4cd7-8c62-a752ff953f09,1171,accepted,genus,Mesotropiscus,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4983450,4NMGW,,5KNTL,,430513f4-35ef-4068-8fee-b4149f1defda,1037,synonym,species,"Pseudepipona lateralis (Fabricius, 1781)","(Fabricius, 1781)",...,,lateralis,,,,,ICZN,,,
4983451,B8W3X,,5KNTM,,5254b6b3-04c9-4981-9252-a3788db3ba3e,1037,ambiguous synonym,species,"Proepipona laterale (Fabricius, 1781)","(Fabricius, 1781)",...,,laterale,,,,,ICZN,,,
4983452,B9M49,,5KNTM,,e2aa97c9-c677-43bd-9b70-0b0089e0840c,1037,ambiguous synonym,species,"Odynerus lateralis (Fabricius, 1781)","(Fabricius, 1781)",...,,lateralis,,,,,ICZN,,,
4983453,4PVZ8,,5KQFS,,54767f71-35d3-4dbf-a576-9ee48c1fbbfe,1037,synonym,species,"Pterochilus glabripalpis de Saussure, 1856","de Saussure, 1856",...,,glabripalpis,,,,,ICZN,,,


We also need the `"parentNameUsageID"` of synonyms, which is necessary the same as the `"acceptedNameUsageID"`. We complete this field as well:

In [10]:
missing_parent = taxon.pandas[(taxon.pandas["parentNameUsageID"] == "") | pd.isna(taxon.pandas["parentNameUsageID"])]
parent2fill = missing_parent[["taxonID", "acceptedNameUsageID"]].merge(
    taxon.pandas[["taxonID", "parentNameUsageID"]],
    how="left",
    left_on="acceptedNameUsageID",
    right_on="taxonID",
)
parent2fill.index = missing_parent.index
taxon.pandas.loc[missing_parent.index, "parentNameUsageID"] = parent2fill.loc[parent2fill.index, "parentNameUsageID"]
taxon.pandas

,taxonID,parentNameUsageID,acceptedNameUsageID,originalNameUsageID,scientificNameID,datasetID,taxonomicStatus,taxonRank,scientificName,scientificNameAuthorship,...,infragenericEpithet,specificEpithet,infraspecificEpithet,cultivarEpithet,nameAccordingTo,namePublishedIn,nomenclaturalCode,nomenclaturalStatus,taxonRemarks,references
0,5T6MX,,5T6MX,,0,,accepted,unranked,Biota,,...,,,,,,,,nomen legitimum,,
1,5SY,HP,5SY,,103a9c28-0651-4695-959f-967f5e2c658c,,accepted,superfamily,Velocipedoidea,,...,,,,,,,,,,
2,3FD8,624LX,3FD8,,e1ae1313-a1f2-4fec-ace6-bba4a245a27c,1171,accepted,genus,Callichilella,,...,,,,,,,,,,
3,63DJ6,624LX,63DJ6,,1c742263-db72-46e5-aa8c-c6f56aa64a35,1171,accepted,genus,Neoloxops,,...,,,,,,,,,,
4,5Q35,624LX,5Q35,,e7eeed09-0d71-4cd7-8c62-a752ff953f09,1171,accepted,genus,Mesotropiscus,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4983450,4NMGW,4MQNP,5KNTL,,430513f4-35ef-4068-8fee-b4149f1defda,1037,synonym,species,"Pseudepipona lateralis (Fabricius, 1781)","(Fabricius, 1781)",...,,lateralis,,,,,ICZN,,,
4983451,B8W3X,4MQNP,5KNTM,,5254b6b3-04c9-4981-9252-a3788db3ba3e,1037,ambiguous synonym,species,"Proepipona laterale (Fabricius, 1781)","(Fabricius, 1781)",...,,laterale,,,,,ICZN,,,
4983452,B9M49,4MQNP,5KNTM,,e2aa97c9-c677-43bd-9b70-0b0089e0840c,1037,ambiguous synonym,species,"Odynerus lateralis (Fabricius, 1781)","(Fabricius, 1781)",...,,lateralis,,,,,ICZN,,,
4983453,4PVZ8,4NTBS,5KQFS,,54767f71-35d3-4dbf-a576-9ee48c1fbbfe,1037,synonym,species,"Pterochilus glabripalpis de Saussure, 1856","de Saussure, 1856",...,,glabripalpis,,,,,ICZN,,,


Checking one extension

In [11]:
col.extensions[0].pandas

Converting to pandas: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2388768/2388768 [00:00<00:00, 2921090.07entry/s]


,taxonID,occurrenceStatus,locationID,locality,countryCode,source
0,6FZX6,native,tdwg:CPP,,,
1,6FZX6,native,tdwg:OFS,,,
2,6FZX6,native,tdwg:GRB,,,
3,6FZX6,native,tdwg:NAT,,,
4,6FZX6,native,tdwg:TVL,,,
...,...,...,...,...,...,...
2388763,8MGDD,native,,Mozambique,,
2388764,8MGDD,native,,Zimbabwe,,
2388765,8MGDD,native,,Botswana,,
2388766,8MGDD,native,,Namibia,,


## Filter chilean species

We are going to filter the species reported in the DwC-A of the COL that are in Chile. To do that, we are going to use a list of the chilean species obtained in [this notebook](<GBIF Chile Species.ipynb>).

In [12]:
chilean_species = pd.read_csv("data/chilean_species.tsv", sep="\t", header=0)
chilean_species

,scientificName,taxonRank,taxonomicStatus
0,"Strongylosoma concolor Gervais, 1847",SPECIES,SYNONYM
1,"Bathyleberis grossmani Kornicker, 1975",SPECIES,ACCEPTED
2,"Neohebestola humeralis (Blanchard, 1851)",SPECIES,ACCEPTED
3,"Bellerus rhianus (Walker, 1842)",SPECIES,ACCEPTED
4,"Ctenoparia topali Smit, 1963",SPECIES,ACCEPTED
...,...,...,...
47414,Oropogon loxensis (Fée) Zukal,SPECIES,ACCEPTED
47415,"Doryssa charpentieri (Reeve, 1859)",SPECIES,ACCEPTED
47416,"Tindaria sulcata (Gould, 1852)",SPECIES,SYNONYM
47417,Ulota fuegiana subsp. darwinii (Mitt.) Malta,SUBSPECIES,SYNONYM


In [13]:
chilean_species.groupby("taxonRank").size()

taxonRank
FORM            149
SPECIES       43813
SUBSPECIES     1448
VARIETY        2009
dtype: int64

In [14]:
chilean_species.groupby(["taxonRank", "taxonomicStatus"]).size().unstack(fill_value=0)

taxonomicStatus,ACCEPTED,DOUBTFUL,SYNONYM
taxonRank,,,
FORM,24,4,121
SPECIES,28140,584,15089
SUBSPECIES,950,8,490
VARIETY,561,19,1429


We take advantage of the [`filter_by_species`](https://pydwca.readthedocs.io/en/latest/dwca.classes.html#dwca.classes.taxon.Taxon.filter_by_species) method of the `Taxon` class of PyDwCA library. This will left all species in a list, its synonyms and taxa parent.

In [15]:
taxon.filter_by_species(chilean_species["scientificName"])
taxon.pandas

Filtering Species: 100%|█████████████████████████████████████████████████████████████████████████████| 100/100 [01:15<00:00,  1.32it/s, Exact match found=33372, Synonyms found=145986, Parents found=36833, Total filtered=212348]


,taxonID,parentNameUsageID,acceptedNameUsageID,originalNameUsageID,scientificNameID,datasetID,taxonomicStatus,taxonRank,scientificName,scientificNameAuthorship,...,infragenericEpithet,specificEpithet,infraspecificEpithet,cultivarEpithet,nameAccordingTo,namePublishedIn,nomenclaturalCode,nomenclaturalStatus,taxonRemarks,references
0,5T6MX,,5T6MX,,0,,accepted,unranked,Biota,,...,,,,,,,,nomen legitimum,,
26,5M78M,62XTM,998SR,,778266e3-5818-4d58-af16-f868ca862149,1141,synonym,variety,Anagallis arvensis var. latifolia (L.) Lange,(L.) Lange,...,,arvensis,latifolia,,,Lange. (1863). In: Pug. Pl. Hispan.: 221.,ICN,,,
42,43HK,624LX,43HK,,845ab74c-f98f-4bbd-9172-bdc3ca854cd7,1171,accepted,genus,Derophthalma,,...,,,,,,,,,,
64,63GPQ,624LX,63GPQ,,ecaf98e7-5e77-4ec5-b96f-9a158477f23d,1171,accepted,genus,Osornocoris,,...,,,,,,,,,,
120,6VKD,624LX,6VKD,,c71d9a67-3632-4011-b4ba-66642ae88516,1171,accepted,genus,Prepops,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4982425,6SGR8,6BNJ,4BPSG,,74f310f0-9fb2-489a-8df9-32649a737c2a,1037,synonym,species,"Odynerus simplicicornis de Saussure, 1854","de Saussure, 1854",...,,simplicicornis,,,,,ICZN,,,
4982487,5FP5F,F3MS,5FP5F,,6f3df4b9-7951-4e60-9a5f-969c390d2e32,1037,accepted,subspecies,Antodynerus sheffieldi bellatuloides Giordani ...,"Giordani Soika, 1990",...,,sheffieldi,bellatuloides,,,,ICZN,,,http://www.zobodat.at
4983059,36TKQ,6XPD,4N56P,,ba2b889c-62d5-4a16-bc41-3abfcee8ce2d,1037,synonym,species,"Discoelius chilensis de Saussure, 1852","de Saussure, 1852",...,,chilensis,,,,,ICZN,,,
4983060,36TLC,6XPD,4N56Q,,c45917ae-30d4-4c2d-a9e7-ae02d581375d,1037,synonym,species,"Discoelius valdiviensis Giordani Soika, 1958","Giordani Soika, 1958",...,,valdiviensis,,,,,ICZN,,,


And check one extension to see is the filter propagates:

In [16]:
col.extensions[0].pandas

,taxonID,occurrenceStatus,locationID,locality,countryCode,source
159,6FZV5,native,tdwg:AGW,,,
160,6FZV5,native,tdwg:BOL,,,
161,6FZV5,native,tdwg:CLN,,,
162,6FZV5,native,tdwg:CLM,,,
163,6FZV5,native,tdwg:ECU,,,
...,...,...,...,...,...,...
2378076,6NTPB,alien,,Peru,,
2378077,6NTPB,alien,,USA,,
2378078,6NTPB,alien,,Canada,,
2378079,6NTPB,alien,,Alaska,,


### Adding needed fields

We need more fields in this dataset, so we are going to added using the [`add_field`]() method.

In [17]:
from tqdm import tqdm

In [18]:
taxon.fields

['http://rs.tdwg.org/dwc/terms/taxonID',
 'http://rs.tdwg.org/dwc/terms/parentNameUsageID',
 'http://rs.tdwg.org/dwc/terms/acceptedNameUsageID',
 'http://rs.tdwg.org/dwc/terms/originalNameUsageID',
 'http://rs.tdwg.org/dwc/terms/scientificNameID',
 'http://rs.tdwg.org/dwc/terms/datasetID',
 'http://rs.tdwg.org/dwc/terms/taxonomicStatus',
 'http://rs.tdwg.org/dwc/terms/taxonRank',
 'http://rs.tdwg.org/dwc/terms/scientificName',
 'http://rs.tdwg.org/dwc/terms/scientificNameAuthorship',
 'https://terms.catalogueoflife.org/notho',
 'http://rs.tdwg.org/dwc/terms/genericName',
 'http://rs.tdwg.org/dwc/terms/infragenericEpithet',
 'http://rs.tdwg.org/dwc/terms/specificEpithet',
 'http://rs.tdwg.org/dwc/terms/infraspecificEpithet',
 'http://rs.tdwg.org/dwc/terms/cultivarEpithet',
 'http://rs.tdwg.org/dwc/terms/nameAccordingTo',
 'http://rs.tdwg.org/dwc/terms/namePublishedIn',
 'http://rs.tdwg.org/dwc/terms/nomenclaturalCode',
 'http://rs.tdwg.org/dwc/terms/nomenclaturalStatus',
 'http://rs.tdw

In [19]:
from dwca.terms import Kingdom, Phylum, DWCClass, Order, Family, Genus

taxon.add_field(Kingdom(index=22))
taxon.add_field(Phylum(index=23))
taxon.add_field(DWCClass(index=24))
taxon.add_field(Order(index=25))
taxon.add_field(Family(index=26))
taxon.add_field(Genus(index=27))

In [20]:
taxon.fields

['http://rs.tdwg.org/dwc/terms/taxonID',
 'http://rs.tdwg.org/dwc/terms/parentNameUsageID',
 'http://rs.tdwg.org/dwc/terms/acceptedNameUsageID',
 'http://rs.tdwg.org/dwc/terms/originalNameUsageID',
 'http://rs.tdwg.org/dwc/terms/scientificNameID',
 'http://rs.tdwg.org/dwc/terms/datasetID',
 'http://rs.tdwg.org/dwc/terms/taxonomicStatus',
 'http://rs.tdwg.org/dwc/terms/taxonRank',
 'http://rs.tdwg.org/dwc/terms/scientificName',
 'http://rs.tdwg.org/dwc/terms/scientificNameAuthorship',
 'https://terms.catalogueoflife.org/notho',
 'http://rs.tdwg.org/dwc/terms/genericName',
 'http://rs.tdwg.org/dwc/terms/infragenericEpithet',
 'http://rs.tdwg.org/dwc/terms/specificEpithet',
 'http://rs.tdwg.org/dwc/terms/infraspecificEpithet',
 'http://rs.tdwg.org/dwc/terms/cultivarEpithet',
 'http://rs.tdwg.org/dwc/terms/nameAccordingTo',
 'http://rs.tdwg.org/dwc/terms/namePublishedIn',
 'http://rs.tdwg.org/dwc/terms/nomenclaturalCode',
 'http://rs.tdwg.org/dwc/terms/nomenclaturalStatus',
 'http://rs.tdw

In [21]:
taxon_df = taxon.pandas
taxon_df

Converting to pandas: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 212348/212348 [00:00<00:00, 1161060.81entry/s]


,taxonID,parentNameUsageID,acceptedNameUsageID,originalNameUsageID,scientificNameID,datasetID,taxonomicStatus,taxonRank,scientificName,scientificNameAuthorship,...,nomenclaturalCode,nomenclaturalStatus,taxonRemarks,references,kingdom,phylum,class,order,family,genus
0,5T6MX,,5T6MX,,0,,accepted,unranked,Biota,,...,,nomen legitimum,,,None,None,None,None,None,None
1,5M78M,62XTM,998SR,,778266e3-5818-4d58-af16-f868ca862149,1141,synonym,variety,Anagallis arvensis var. latifolia (L.) Lange,(L.) Lange,...,ICN,,,,None,None,None,None,None,None
2,43HK,624LX,43HK,,845ab74c-f98f-4bbd-9172-bdc3ca854cd7,1171,accepted,genus,Derophthalma,,...,,,,,None,None,None,None,None,None
3,63GPQ,624LX,63GPQ,,ecaf98e7-5e77-4ec5-b96f-9a158477f23d,1171,accepted,genus,Osornocoris,,...,,,,,None,None,None,None,None,None
4,6VKD,624LX,6VKD,,c71d9a67-3632-4011-b4ba-66642ae88516,1171,accepted,genus,Prepops,,...,,,,,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212343,6SGR8,6BNJ,4BPSG,,74f310f0-9fb2-489a-8df9-32649a737c2a,1037,synonym,species,"Odynerus simplicicornis de Saussure, 1854","de Saussure, 1854",...,ICZN,,,,None,None,None,None,None,None
212344,5FP5F,F3MS,5FP5F,,6f3df4b9-7951-4e60-9a5f-969c390d2e32,1037,accepted,subspecies,Antodynerus sheffieldi bellatuloides Giordani ...,"Giordani Soika, 1990",...,ICZN,,,http://www.zobodat.at,None,None,None,None,None,None
212345,36TKQ,6XPD,4N56P,,ba2b889c-62d5-4a16-bc41-3abfcee8ce2d,1037,synonym,species,"Discoelius chilensis de Saussure, 1852","de Saussure, 1852",...,ICZN,,,,None,None,None,None,None,None
212346,36TLC,6XPD,4N56Q,,c45917ae-30d4-4c2d-a9e7-ae02d581375d,1037,synonym,species,"Discoelius valdiviensis Giordani Soika, 1958","Giordani Soika, 1958",...,ICZN,,,,None,None,None,None,None,None


And, we are going to take advantage of the pandas `DataFrame` to implement an algorithm that are going to populate those field. The idea is:
1. Find taxonRank corresponding to the field, e.g. `"kindgom"` for `Kingdom`.
2. All child taxa of this has the `"scientificName"` found as that field. Child taxa are the taxa in with `"parentNameUsageID"` is the taxa found.
3. Child of the above has also  that `"scientificName"` as the field.
4. Repeat until there are no more childs.

In [22]:
def fill_taxa(df: pd.DataFrame, rank_name: str, to_print: bool = True) -> None:
    ranks = taxon_df[taxon_df["taxonRank"] == rank_name]["scientificName"]
    if to_print:
        print(ranks)
    else:
        bar_progress = tqdm(f"Working with {rank_name}: {ranks.iloc[0]}", total=len(ranks))
    for rank in ranks:
        total = len(taxon_df)
        if to_print:
            bar_progress = tqdm(desc=f"Working with {rank_name}: {rank}", total=total)
        else:
            bar_progress.set_description(f"Working with {rank_name}: {rank}", refresh=False)
        posfix = dict()
        childs = taxon_df[
            (taxon_df["taxonRank"] == rank_name) &
            (taxon_df["scientificName"] == rank)
        ]
        while len(childs) > 0:
            taxon_df.loc[childs.index, rank_name] = rank
            total -= len(childs)
            if to_print:
                bar_progress.update(n=len(childs))
            taxa = childs["taxonID"]
            posfix["Working with ranks"] = ", ".join([str(rank) for rank in pd.unique(childs["taxonRank"])])
            bar_progress.set_postfix(ordered_dict=posfix, refresh=to_print)
            childs = taxon_df[taxon_df["parentNameUsageID"].isin(taxa)]
            synonyms = taxon.all_synonyms(childs["taxonID"])
            childs = taxon_df[
                (taxon_df["parentNameUsageID"].isin(taxa)) |
                (taxon_df["taxonID"].isin(synonyms))
            ]
        if to_print:
            bar_progress.update(n=total)
            bar_progress.close()
        else:
            bar_progress.update(n=1)
    if not to_print:
        bar_progress.close()
    return

In [23]:
fill_taxa(taxon_df, "kingdom")
taxon_df

1716     Animalia
7738      Plantae
46999       Fungi
Name: scientificName, dtype: object


Working with kingdom: Fungi: 100%|████████████████████████████████████████████████| 212348/212348 [00:01<00:00, 181118.65it/s, Working with ranks=species, variety, form, proles, subspecies, other, subform, unranked, subvariety]


,taxonID,parentNameUsageID,acceptedNameUsageID,originalNameUsageID,scientificNameID,datasetID,taxonomicStatus,taxonRank,scientificName,scientificNameAuthorship,...,nomenclaturalCode,nomenclaturalStatus,taxonRemarks,references,kingdom,phylum,class,order,family,genus
0,5T6MX,,5T6MX,,0,,accepted,unranked,Biota,,...,,nomen legitimum,,,None,None,None,None,None,None
1,5M78M,62XTM,998SR,,778266e3-5818-4d58-af16-f868ca862149,1141,synonym,variety,Anagallis arvensis var. latifolia (L.) Lange,(L.) Lange,...,ICN,,,,Plantae,None,None,None,None,None
2,43HK,624LX,43HK,,845ab74c-f98f-4bbd-9172-bdc3ca854cd7,1171,accepted,genus,Derophthalma,,...,,,,,Animalia,None,None,None,None,None
3,63GPQ,624LX,63GPQ,,ecaf98e7-5e77-4ec5-b96f-9a158477f23d,1171,accepted,genus,Osornocoris,,...,,,,,Animalia,None,None,None,None,None
4,6VKD,624LX,6VKD,,c71d9a67-3632-4011-b4ba-66642ae88516,1171,accepted,genus,Prepops,,...,,,,,Animalia,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212343,6SGR8,6BNJ,4BPSG,,74f310f0-9fb2-489a-8df9-32649a737c2a,1037,synonym,species,"Odynerus simplicicornis de Saussure, 1854","de Saussure, 1854",...,ICZN,,,,Animalia,None,None,None,None,None
212344,5FP5F,F3MS,5FP5F,,6f3df4b9-7951-4e60-9a5f-969c390d2e32,1037,accepted,subspecies,Antodynerus sheffieldi bellatuloides Giordani ...,"Giordani Soika, 1990",...,ICZN,,,http://www.zobodat.at,Animalia,None,None,None,None,None
212345,36TKQ,6XPD,4N56P,,ba2b889c-62d5-4a16-bc41-3abfcee8ce2d,1037,synonym,species,"Discoelius chilensis de Saussure, 1852","de Saussure, 1852",...,ICZN,,,,Animalia,None,None,None,None,None
212346,36TLC,6XPD,4N56Q,,c45917ae-30d4-4c2d-a9e7-ae02d581375d,1037,synonym,species,"Discoelius valdiviensis Giordani Soika, 1958","Giordani Soika, 1958",...,ICZN,,,,Animalia,None,None,None,None,None


In [24]:
fill_taxa(taxon_df, "phylum")
taxon_df

5                                             Echinodermata
5055                                               Chordata
6874                                      Bryophyta Schimp.
6875                Marchantiophyta Stotler & Crand.-Stotl.
6876               Anthocerotophyta Stotler & Crand.-Stotl.
7110                                             Arthropoda
7886                                           Tracheophyta
7921                                               Annelida
7923                                               Cnidaria
47000                                         Basidiomycota
47001                                            Ascomycota
47002                                   Entomophthoromycota
47003                                            Zygomycota
47004                                         Zoopagomycota
47005                                          Mucoromycota
47006                                       Chytridiomycota
54290                                   

Working with phylum: Bryophyta Schimp.: 100%|███████████████████████████████████████████████████████████████████████████████████| 212348/212348 [00:01<00:00, 172585.08it/s, Working with ranks=variety, subspecies, species, form]
Working with phylum: Marchantiophyta Stotler & Crand.-Stotl.: 100%|█████████████████████████████████████████████████████████████| 212348/212348 [00:01<00:00, 184474.11it/s, Working with ranks=subspecies, variety, species, form]
Working with phylum: Anthocerotophyta Stotler & Crand.-Stotl.: 100%|████████████████████████████████████████████████████████████| 212348/212348 [00:00<00:00, 225364.80it/s, Working with ranks=species, variety, subspecies, form]
Working with phylum: Phoronida: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 212348/212348 [00:00<00:00, 671479.74it/s, Working with ranks=species]
Working with phylum: Ctenophora Eschscholtz, 1829: 100%|████████████████████████████████

,taxonID,parentNameUsageID,acceptedNameUsageID,originalNameUsageID,scientificNameID,datasetID,taxonomicStatus,taxonRank,scientificName,scientificNameAuthorship,...,nomenclaturalCode,nomenclaturalStatus,taxonRemarks,references,kingdom,phylum,class,order,family,genus
0,5T6MX,,5T6MX,,0,,accepted,unranked,Biota,,...,,nomen legitimum,,,None,None,None,None,None,None
1,5M78M,62XTM,998SR,,778266e3-5818-4d58-af16-f868ca862149,1141,synonym,variety,Anagallis arvensis var. latifolia (L.) Lange,(L.) Lange,...,ICN,,,,Plantae,Tracheophyta,None,None,None,None
2,43HK,624LX,43HK,,845ab74c-f98f-4bbd-9172-bdc3ca854cd7,1171,accepted,genus,Derophthalma,,...,,,,,Animalia,Arthropoda,None,None,None,None
3,63GPQ,624LX,63GPQ,,ecaf98e7-5e77-4ec5-b96f-9a158477f23d,1171,accepted,genus,Osornocoris,,...,,,,,Animalia,Arthropoda,None,None,None,None
4,6VKD,624LX,6VKD,,c71d9a67-3632-4011-b4ba-66642ae88516,1171,accepted,genus,Prepops,,...,,,,,Animalia,Arthropoda,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212343,6SGR8,6BNJ,4BPSG,,74f310f0-9fb2-489a-8df9-32649a737c2a,1037,synonym,species,"Odynerus simplicicornis de Saussure, 1854","de Saussure, 1854",...,ICZN,,,,Animalia,Arthropoda,None,None,None,None
212344,5FP5F,F3MS,5FP5F,,6f3df4b9-7951-4e60-9a5f-969c390d2e32,1037,accepted,subspecies,Antodynerus sheffieldi bellatuloides Giordani ...,"Giordani Soika, 1990",...,ICZN,,,http://www.zobodat.at,Animalia,Arthropoda,None,None,None,None
212345,36TKQ,6XPD,4N56P,,ba2b889c-62d5-4a16-bc41-3abfcee8ce2d,1037,synonym,species,"Discoelius chilensis de Saussure, 1852","de Saussure, 1852",...,ICZN,,,,Animalia,Arthropoda,None,None,None,None
212346,36TLC,6XPD,4N56Q,,c45917ae-30d4-4c2d-a9e7-ae02d581375d,1037,synonym,species,"Discoelius valdiviensis Giordani Soika, 1958","Giordani Soika, 1958",...,ICZN,,,,Animalia,Arthropoda,None,None,None,None


In [25]:
fill_taxa(taxon_df, "class", to_print=False)
taxon_df

Working with class: Echinoidea Schumacher, 1817: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 133/133 [01:50<00:00,  1.20it/s, Working with ranks=species]


,taxonID,parentNameUsageID,acceptedNameUsageID,originalNameUsageID,scientificNameID,datasetID,taxonomicStatus,taxonRank,scientificName,scientificNameAuthorship,...,nomenclaturalCode,nomenclaturalStatus,taxonRemarks,references,kingdom,phylum,class,order,family,genus
0,5T6MX,,5T6MX,,0,,accepted,unranked,Biota,,...,,nomen legitimum,,,None,None,None,None,None,None
1,5M78M,62XTM,998SR,,778266e3-5818-4d58-af16-f868ca862149,1141,synonym,variety,Anagallis arvensis var. latifolia (L.) Lange,(L.) Lange,...,ICN,,,,Plantae,Tracheophyta,Magnoliopsida,None,None,None
2,43HK,624LX,43HK,,845ab74c-f98f-4bbd-9172-bdc3ca854cd7,1171,accepted,genus,Derophthalma,,...,,,,,Animalia,Arthropoda,Insecta,None,None,None
3,63GPQ,624LX,63GPQ,,ecaf98e7-5e77-4ec5-b96f-9a158477f23d,1171,accepted,genus,Osornocoris,,...,,,,,Animalia,Arthropoda,Insecta,None,None,None
4,6VKD,624LX,6VKD,,c71d9a67-3632-4011-b4ba-66642ae88516,1171,accepted,genus,Prepops,,...,,,,,Animalia,Arthropoda,Insecta,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212343,6SGR8,6BNJ,4BPSG,,74f310f0-9fb2-489a-8df9-32649a737c2a,1037,synonym,species,"Odynerus simplicicornis de Saussure, 1854","de Saussure, 1854",...,ICZN,,,,Animalia,Arthropoda,Insecta,None,None,None
212344,5FP5F,F3MS,5FP5F,,6f3df4b9-7951-4e60-9a5f-969c390d2e32,1037,accepted,subspecies,Antodynerus sheffieldi bellatuloides Giordani ...,"Giordani Soika, 1990",...,ICZN,,,http://www.zobodat.at,Animalia,Arthropoda,Insecta,None,None,None
212345,36TKQ,6XPD,4N56P,,ba2b889c-62d5-4a16-bc41-3abfcee8ce2d,1037,synonym,species,"Discoelius chilensis de Saussure, 1852","de Saussure, 1852",...,ICZN,,,,Animalia,Arthropoda,Insecta,None,None,None
212346,36TLC,6XPD,4N56Q,,c45917ae-30d4-4c2d-a9e7-ae02d581375d,1037,synonym,species,"Discoelius valdiviensis Giordani Soika, 1958","Giordani Soika, 1958",...,ICZN,,,,Animalia,Arthropoda,Insecta,None,None,None


In [26]:
fill_taxa(taxon_df, "order", to_print=False)
taxon_df

Working with order: Tanaidacea Dana, 1849: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 640/640 [06:33<00:00,  1.63it/s, Working with ranks=species]


,taxonID,parentNameUsageID,acceptedNameUsageID,originalNameUsageID,scientificNameID,datasetID,taxonomicStatus,taxonRank,scientificName,scientificNameAuthorship,...,nomenclaturalCode,nomenclaturalStatus,taxonRemarks,references,kingdom,phylum,class,order,family,genus
0,5T6MX,,5T6MX,,0,,accepted,unranked,Biota,,...,,nomen legitimum,,,None,None,None,None,None,None
1,5M78M,62XTM,998SR,,778266e3-5818-4d58-af16-f868ca862149,1141,synonym,variety,Anagallis arvensis var. latifolia (L.) Lange,(L.) Lange,...,ICN,,,,Plantae,Tracheophyta,Magnoliopsida,Ericales,None,None
2,43HK,624LX,43HK,,845ab74c-f98f-4bbd-9172-bdc3ca854cd7,1171,accepted,genus,Derophthalma,,...,,,,,Animalia,Arthropoda,Insecta,Hemiptera,None,None
3,63GPQ,624LX,63GPQ,,ecaf98e7-5e77-4ec5-b96f-9a158477f23d,1171,accepted,genus,Osornocoris,,...,,,,,Animalia,Arthropoda,Insecta,Hemiptera,None,None
4,6VKD,624LX,6VKD,,c71d9a67-3632-4011-b4ba-66642ae88516,1171,accepted,genus,Prepops,,...,,,,,Animalia,Arthropoda,Insecta,Hemiptera,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212343,6SGR8,6BNJ,4BPSG,,74f310f0-9fb2-489a-8df9-32649a737c2a,1037,synonym,species,"Odynerus simplicicornis de Saussure, 1854","de Saussure, 1854",...,ICZN,,,,Animalia,Arthropoda,Insecta,Hymenoptera,None,None
212344,5FP5F,F3MS,5FP5F,,6f3df4b9-7951-4e60-9a5f-969c390d2e32,1037,accepted,subspecies,Antodynerus sheffieldi bellatuloides Giordani ...,"Giordani Soika, 1990",...,ICZN,,,http://www.zobodat.at,Animalia,Arthropoda,Insecta,Hymenoptera,None,None
212345,36TKQ,6XPD,4N56P,,ba2b889c-62d5-4a16-bc41-3abfcee8ce2d,1037,synonym,species,"Discoelius chilensis de Saussure, 1852","de Saussure, 1852",...,ICZN,,,,Animalia,Arthropoda,Insecta,Hymenoptera,None,None
212346,36TLC,6XPD,4N56Q,,c45917ae-30d4-4c2d-a9e7-ae02d581375d,1037,synonym,species,"Discoelius valdiviensis Giordani Soika, 1958","Giordani Soika, 1958",...,ICZN,,,,Animalia,Arthropoda,Insecta,Hymenoptera,None,None


In [27]:
fill_taxa(taxon_df, "family", to_print=False)
taxon_df

Working with family: Eumenidae: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3198/3198 [21:52<00:00,  2.44it/s, Working with ranks=subspecies, species]


,taxonID,parentNameUsageID,acceptedNameUsageID,originalNameUsageID,scientificNameID,datasetID,taxonomicStatus,taxonRank,scientificName,scientificNameAuthorship,...,nomenclaturalCode,nomenclaturalStatus,taxonRemarks,references,kingdom,phylum,class,order,family,genus
0,5T6MX,,5T6MX,,0,,accepted,unranked,Biota,,...,,nomen legitimum,,,None,None,None,None,None,None
1,5M78M,62XTM,998SR,,778266e3-5818-4d58-af16-f868ca862149,1141,synonym,variety,Anagallis arvensis var. latifolia (L.) Lange,(L.) Lange,...,ICN,,,,Plantae,Tracheophyta,Magnoliopsida,Ericales,Primulaceae Batsch ex Borkh.,None
2,43HK,624LX,43HK,,845ab74c-f98f-4bbd-9172-bdc3ca854cd7,1171,accepted,genus,Derophthalma,,...,,,,,Animalia,Arthropoda,Insecta,Hemiptera,Miridae,None
3,63GPQ,624LX,63GPQ,,ecaf98e7-5e77-4ec5-b96f-9a158477f23d,1171,accepted,genus,Osornocoris,,...,,,,,Animalia,Arthropoda,Insecta,Hemiptera,Miridae,None
4,6VKD,624LX,6VKD,,c71d9a67-3632-4011-b4ba-66642ae88516,1171,accepted,genus,Prepops,,...,,,,,Animalia,Arthropoda,Insecta,Hemiptera,Miridae,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212343,6SGR8,6BNJ,4BPSG,,74f310f0-9fb2-489a-8df9-32649a737c2a,1037,synonym,species,"Odynerus simplicicornis de Saussure, 1854","de Saussure, 1854",...,ICZN,,,,Animalia,Arthropoda,Insecta,Hymenoptera,Eumenidae,None
212344,5FP5F,F3MS,5FP5F,,6f3df4b9-7951-4e60-9a5f-969c390d2e32,1037,accepted,subspecies,Antodynerus sheffieldi bellatuloides Giordani ...,"Giordani Soika, 1990",...,ICZN,,,http://www.zobodat.at,Animalia,Arthropoda,Insecta,Hymenoptera,Eumenidae,None
212345,36TKQ,6XPD,4N56P,,ba2b889c-62d5-4a16-bc41-3abfcee8ce2d,1037,synonym,species,"Discoelius chilensis de Saussure, 1852","de Saussure, 1852",...,ICZN,,,,Animalia,Arthropoda,Insecta,Hymenoptera,Eumenidae,None
212346,36TLC,6XPD,4N56Q,,c45917ae-30d4-4c2d-a9e7-ae02d581375d,1037,synonym,species,"Discoelius valdiviensis Giordani Soika, 1958","Giordani Soika, 1958",...,ICZN,,,,Animalia,Arthropoda,Insecta,Hymenoptera,Eumenidae,None


In [28]:
fill_taxa(taxon_df, "genus", to_print=False)
taxon_df

Working with genus: Hypodynerus: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16423/16423 [1:07:26<00:00,  4.06it/s, Working with ranks=species]


,taxonID,parentNameUsageID,acceptedNameUsageID,originalNameUsageID,scientificNameID,datasetID,taxonomicStatus,taxonRank,scientificName,scientificNameAuthorship,...,nomenclaturalCode,nomenclaturalStatus,taxonRemarks,references,kingdom,phylum,class,order,family,genus
0,5T6MX,,5T6MX,,0,,accepted,unranked,Biota,,...,,nomen legitimum,,,None,None,None,None,None,None
1,5M78M,62XTM,998SR,,778266e3-5818-4d58-af16-f868ca862149,1141,synonym,variety,Anagallis arvensis var. latifolia (L.) Lange,(L.) Lange,...,ICN,,,,Plantae,Tracheophyta,Magnoliopsida,Ericales,Primulaceae Batsch ex Borkh.,Lysimachia Tourn. ex L.
2,43HK,624LX,43HK,,845ab74c-f98f-4bbd-9172-bdc3ca854cd7,1171,accepted,genus,Derophthalma,,...,,,,,Animalia,Arthropoda,Insecta,Hemiptera,Miridae,Derophthalma
3,63GPQ,624LX,63GPQ,,ecaf98e7-5e77-4ec5-b96f-9a158477f23d,1171,accepted,genus,Osornocoris,,...,,,,,Animalia,Arthropoda,Insecta,Hemiptera,Miridae,Osornocoris
4,6VKD,624LX,6VKD,,c71d9a67-3632-4011-b4ba-66642ae88516,1171,accepted,genus,Prepops,,...,,,,,Animalia,Arthropoda,Insecta,Hemiptera,Miridae,Prepops
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212343,6SGR8,6BNJ,4BPSG,,74f310f0-9fb2-489a-8df9-32649a737c2a,1037,synonym,species,"Odynerus simplicicornis de Saussure, 1854","de Saussure, 1854",...,ICZN,,,,Animalia,Arthropoda,Insecta,Hymenoptera,Eumenidae,Pachodynerus
212344,5FP5F,F3MS,5FP5F,,6f3df4b9-7951-4e60-9a5f-969c390d2e32,1037,accepted,subspecies,Antodynerus sheffieldi bellatuloides Giordani ...,"Giordani Soika, 1990",...,ICZN,,,http://www.zobodat.at,Animalia,Arthropoda,Insecta,Hymenoptera,Eumenidae,Antodynerus
212345,36TKQ,6XPD,4N56P,,ba2b889c-62d5-4a16-bc41-3abfcee8ce2d,1037,synonym,species,"Discoelius chilensis de Saussure, 1852","de Saussure, 1852",...,ICZN,,,,Animalia,Arthropoda,Insecta,Hymenoptera,Eumenidae,Protodiscoelius
212346,36TLC,6XPD,4N56Q,,c45917ae-30d4-4c2d-a9e7-ae02d581375d,1037,synonym,species,"Discoelius valdiviensis Giordani Soika, 1958","Giordani Soika, 1958",...,ICZN,,,,Animalia,Arthropoda,Insecta,Hymenoptera,Eumenidae,Protodiscoelius


In [29]:
taxon.pandas = taxon_df
print(col)
print(col.extensions)

col-clb-9910 [Core: http://rs.tdwg.org/dwc/terms/Taxon, Entries: 212348]
[<XMLObject tag=extension>, <XMLObject tag=extension>, <XMLObject tag=extension>]


We save this modified archive as [`chilean-col.zip`](data/chilean-col.zip)

In [30]:
col.to_file("data/chilean-col.zip")

Writing data http://rs.tdwg.org/dwc/terms/Taxon: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 212348/212348 [00:16<00:00, 12684.75line/s]
Writing data http://rs.gbif.org/terms/1.0/Distribution: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 137733/137733 [00:02<00:00, 61549.48line/s]
Writing data http://rs.gbif.org/terms/1.0/SpeciesProfile: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 33820/33820 [00:00<00:00, 59676.76line/s]
Writing data http://rs.gbif.org/terms/1.0/VernacularName: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 37868/37868 [00:00<00:00, 96536.64line/s]


In [31]:
del col

## Extracting species

For our pipeline, we need to extract the tracheophytas. First, we check if this is directly on the dataset. We start reading the Darwin Core Archive generated on the previous step:

In [32]:
chilean_col = DarwinCoreArchive.from_file("data/chilean-col.zip")

/home/ubuntu/pydwca/src/dwca/classes/data_file.py:198: UserWarning: https://terms.catalogueoflife.org/notho not in expected namespace for http://rs.tdwg.org/dwc/terms/Taxon class. Some functionalities may not be available.
  warn(f"{element.get('term')} not in expected namespace for "
/home/ubuntu/pydwca/src/dwca/base/darwincore_archive.py:76: UserWarning: http://rs.gbif.org/terms/1.0/Distribution not in expected namespace. Some functionalities may not be available.
  warn(f"{element.get('rowType')} not in expected namespace. "
/home/ubuntu/pydwca/src/dwca/base/darwincore_archive.py:76: UserWarning: http://rs.gbif.org/terms/1.0/SpeciesProfile not in expected namespace. Some functionalities may not be available.
  warn(f"{element.get('rowType')} not in expected namespace. "
/home/ubuntu/pydwca/src/dwca/base/darwincore_archive.py:76: UserWarning: http://rs.gbif.org/terms/1.0/VernacularName not in expected namespace. Some functionalities may not be available.
  warn(f"{element.get('rowTyp

In [33]:
taxon = chilean_col.core
taxon.pandas

Converting to pandas: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 212348/212348 [00:00<00:00, 1004940.99entry/s]


,taxonID,parentNameUsageID,acceptedNameUsageID,originalNameUsageID,scientificNameID,datasetID,taxonomicStatus,taxonRank,scientificName,scientificNameAuthorship,...,nomenclaturalCode,nomenclaturalStatus,taxonRemarks,references,kingdom,phylum,class,order,family,genus
0,5T6MX,,5T6MX,,0,,accepted,unranked,Biota,,...,,nomen legitimum,,,,,,,,
1,5M78M,62XTM,998SR,,778266e3-5818-4d58-af16-f868ca862149,1141,synonym,variety,Anagallis arvensis var. latifolia (L.) Lange,(L.) Lange,...,ICN,,,,Plantae,Tracheophyta,Magnoliopsida,Ericales,Primulaceae Batsch ex Borkh.,Lysimachia Tourn. ex L.
2,43HK,624LX,43HK,,845ab74c-f98f-4bbd-9172-bdc3ca854cd7,1171,accepted,genus,Derophthalma,,...,,,,,Animalia,Arthropoda,Insecta,Hemiptera,Miridae,Derophthalma
3,63GPQ,624LX,63GPQ,,ecaf98e7-5e77-4ec5-b96f-9a158477f23d,1171,accepted,genus,Osornocoris,,...,,,,,Animalia,Arthropoda,Insecta,Hemiptera,Miridae,Osornocoris
4,6VKD,624LX,6VKD,,c71d9a67-3632-4011-b4ba-66642ae88516,1171,accepted,genus,Prepops,,...,,,,,Animalia,Arthropoda,Insecta,Hemiptera,Miridae,Prepops
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212343,6SGR8,6BNJ,4BPSG,,74f310f0-9fb2-489a-8df9-32649a737c2a,1037,synonym,species,"Odynerus simplicicornis de Saussure, 1854","de Saussure, 1854",...,ICZN,,,,Animalia,Arthropoda,Insecta,Hymenoptera,Eumenidae,Pachodynerus
212344,5FP5F,F3MS,5FP5F,,6f3df4b9-7951-4e60-9a5f-969c390d2e32,1037,accepted,subspecies,Antodynerus sheffieldi bellatuloides Giordani ...,"Giordani Soika, 1990",...,ICZN,,,http://www.zobodat.at,Animalia,Arthropoda,Insecta,Hymenoptera,Eumenidae,Antodynerus
212345,36TKQ,6XPD,4N56P,,ba2b889c-62d5-4a16-bc41-3abfcee8ce2d,1037,synonym,species,"Discoelius chilensis de Saussure, 1852","de Saussure, 1852",...,ICZN,,,,Animalia,Arthropoda,Insecta,Hymenoptera,Eumenidae,Protodiscoelius
212346,36TLC,6XPD,4N56Q,,c45917ae-30d4-4c2d-a9e7-ae02d581375d,1037,synonym,species,"Discoelius valdiviensis Giordani Soika, 1958","Giordani Soika, 1958",...,ICZN,,,,Animalia,Arthropoda,Insecta,Hymenoptera,Eumenidae,Protodiscoelius


In [34]:
taxon.pandas[taxon.pandas["scientificName"].str.lower() == "tracheophyta"]

,taxonID,parentNameUsageID,acceptedNameUsageID,originalNameUsageID,scientificNameID,datasetID,taxonomicStatus,taxonRank,scientificName,scientificNameAuthorship,...,nomenclaturalCode,nomenclaturalStatus,taxonRemarks,references,kingdom,phylum,class,order,family,genus
7886,TP,P,TP,,bfb709db-491c-48b6-80f5-32ef14f63e4f,,accepted,phylum,Tracheophyta,,...,,nomen legitimum,,,Plantae,Tracheophyta,,,,


In [35]:
taxon.pandas.groupby("phylum").size()

phylum
                                                                                                               12
Acanthocephala Rudolphi, 1802                                                                                  57
Annelida                                                                                                     2252
Anthocerotophyta Stotler & Crand.-Stotl.                                                                      208
Arthropoda                                                                                                  28824
Ascomycota                                                                                                  14185
Basidiomycota                                                                                                8470
Brachiopoda Duméril, 1805                                                                                     155
Bryophyta Schimp.                                                                

In [36]:
taxon.pandas[taxon.pandas["phylum"] == "Tracheophyta"]

,taxonID,parentNameUsageID,acceptedNameUsageID,originalNameUsageID,scientificNameID,datasetID,taxonomicStatus,taxonRank,scientificName,scientificNameAuthorship,...,nomenclaturalCode,nomenclaturalStatus,taxonRemarks,references,kingdom,phylum,class,order,family,genus
1,5M78M,62XTM,998SR,,778266e3-5818-4d58-af16-f868ca862149,1141,synonym,variety,Anagallis arvensis var. latifolia (L.) Lange,(L.) Lange,...,ICN,,,,Plantae,Tracheophyta,Magnoliopsida,Ericales,Primulaceae Batsch ex Borkh.,Lysimachia Tourn. ex L.
6,5M78L,62XTM,998SR,,6875f1c1-4458-4ff4-9676-0efd313b5c32,1141,ambiguous synonym,variety,Anagallis arvensis var. caerulea (L.) Gouan,(L.) Gouan,...,ICN,,,,Plantae,Tracheophyta,Magnoliopsida,Ericales,Primulaceae Batsch ex Borkh.,Lysimachia Tourn. ex L.
7,B5KLK,62XTM,998SR,,0104e482-45e1-494f-b277-fb36a5a5506b,1141,synonym,variety,Anagallis caerulea var. alba Gray,Gray,...,ICN,,,,Plantae,Tracheophyta,Magnoliopsida,Ericales,Primulaceae Batsch ex Borkh.,Lysimachia Tourn. ex L.
8,9YJT8,62XTM,998SR,,d95e962b-fe0d-4d4b-8e72-509e364fda78,1141,ambiguous synonym,variety,Anagallis arvensis var. caerulea (L.) Arechav.,(L.) Arechav.,...,ICN,,,,Plantae,Tracheophyta,Magnoliopsida,Ericales,Primulaceae Batsch ex Borkh.,Lysimachia Tourn. ex L.
9,5PXCX,62XTM,998SR,,e9a1555c-f3c5-4f94-b812-e1a0e62e76b5,1141,synonym,variety,Lysimachia arvensis var. caerulea (L.) Turland...,(L.) Turland & Bergmeier,...,ICN,,,,Plantae,Tracheophyta,Magnoliopsida,Ericales,Primulaceae Batsch ex Borkh.,Lysimachia Tourn. ex L.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199076,5DH8Z,DX9C,5M88V,,d0921fd5-8b5d-4d88-96fd-521232ab263a,1141,synonym,infraspecific name,Anemone decapetala triloba Juss.,Juss.,...,ICN,,,,Plantae,Tracheophyta,Magnoliopsida,Ranunculales Juss. ex Bercht. & J. Presl,Ranunculaceae Juss.,Anemone L.
199077,5DH92,DX9C,5M88V,,692f61a1-5e7c-48bc-928e-2a23aabbd0bc,1141,synonym,variety,Anemone decapetala var. triternata (Vahl) Kuntze,(Vahl) Kuntze,...,ICN,,,,Plantae,Tracheophyta,Magnoliopsida,Ranunculales Juss. ex Bercht. & J. Presl,Ranunculaceae Juss.,Anemone L.
199078,LJQQ,LJKG,5MJWR,,c477d57a-ce7e-4374-8780-191326393615,1141,synonym,species,Berberis gayi K. Koch,K. Koch,...,ICN,,,,Plantae,Tracheophyta,Magnoliopsida,Ranunculales Juss. ex Bercht. & J. Presl,Berberidaceae Juss.,Berberis L.
199079,LJHV,LJKG,5MJWR,,a36bd40b-14a4-4df4-9de6-ac26edf0a0e2,1141,synonym,species,Berberis brachybotrya Gay,Gay,...,ICN,,,,Plantae,Tracheophyta,Magnoliopsida,Ranunculales Juss. ex Bercht. & J. Presl,Berberidaceae Juss.,Berberis L.


In [37]:
taxon.pandas[taxon.pandas["phylum"] == "Tracheophyta"].groupby("class").size()

class
                      1
Ginkgoopsida         36
Gnetopsida           51
Liliopsida        28162
Lycopodiopsida      313
Magnoliopsida     71044
Pinopsida           350
Polypodiopsida     4297
dtype: int64

Due to Tracheophyta is a phylum, we can extract all phylum except this one:

In [38]:
taxon.fields

['http://rs.tdwg.org/dwc/terms/taxonID',
 'http://rs.tdwg.org/dwc/terms/parentNameUsageID',
 'http://rs.tdwg.org/dwc/terms/acceptedNameUsageID',
 'http://rs.tdwg.org/dwc/terms/originalNameUsageID',
 'http://rs.tdwg.org/dwc/terms/scientificNameID',
 'http://rs.tdwg.org/dwc/terms/datasetID',
 'http://rs.tdwg.org/dwc/terms/taxonomicStatus',
 'http://rs.tdwg.org/dwc/terms/taxonRank',
 'http://rs.tdwg.org/dwc/terms/scientificName',
 'http://rs.tdwg.org/dwc/terms/scientificNameAuthorship',
 'https://terms.catalogueoflife.org/notho',
 'http://rs.tdwg.org/dwc/terms/genericName',
 'http://rs.tdwg.org/dwc/terms/infragenericEpithet',
 'http://rs.tdwg.org/dwc/terms/specificEpithet',
 'http://rs.tdwg.org/dwc/terms/infraspecificEpithet',
 'http://rs.tdwg.org/dwc/terms/cultivarEpithet',
 'http://rs.tdwg.org/dwc/terms/nameAccordingTo',
 'http://rs.tdwg.org/dwc/terms/namePublishedIn',
 'http://rs.tdwg.org/dwc/terms/nomenclaturalCode',
 'http://rs.tdwg.org/dwc/terms/nomenclaturalStatus',
 'http://rs.tdw

In [39]:
found_phyla = pd.unique(taxon.pandas["phylum"])
found_phyla

array(['', 'Tracheophyta', 'Arthropoda', 'Echinodermata', 'Chordata',
       'Cnidaria', 'Bryophyta Schimp.',
       'Marchantiophyta Stotler & Crand.-Stotl.',
       'Anthocerotophyta Stotler & Crand.-Stotl.', 'Annelida',
       'Platyhelminthes Minot, 1876', 'Ascomycota', 'Basidiomycota',
       'Entomophthoromycota', 'Zygomycota', 'Zoopagomycota',
       'Mucoromycota', 'Chytridiomycota', 'Nematomorpha', 'Chaetognatha',
       'Kinorhyncha Reinhard, 1885', 'Rotifera', 'Phoronida', 'Mollusca',
       'Ctenophora Eschscholtz, 1829', 'Brachiopoda Duméril, 1805',
       'Hemichordata Bateson, 1885', 'Stomochordata',
       'Onychophora Grube, 1853', 'Tardigrada Doyère, 1840', 'Sipuncula',
       'Sipunculoidea Sedgwick, 1898', 'Acanthocephala Rudolphi, 1802',
       'Xenacoelomorpha Philippe, Brinkmann, Copley, Moroz, Nakano, Poustka, Wallberg, Peterson & Telford, 2011',
       'Bryozoa', 'Nematoda', 'Nemertea', 'Priapulida Théel, 1906',
       'Porifera Grant, 1836'], dtype=object)

In [40]:
found_phyla = list(found_phyla)
found_phyla.remove("")
found_phyla.remove("Tracheophyta")
found_phyla

['Arthropoda',
 'Echinodermata',
 'Chordata',
 'Cnidaria',
 'Bryophyta Schimp.',
 'Marchantiophyta Stotler & Crand.-Stotl.',
 'Anthocerotophyta Stotler & Crand.-Stotl.',
 'Annelida',
 'Platyhelminthes Minot, 1876',
 'Ascomycota',
 'Basidiomycota',
 'Entomophthoromycota',
 'Zygomycota',
 'Zoopagomycota',
 'Mucoromycota',
 'Chytridiomycota',
 'Nematomorpha',
 'Chaetognatha',
 'Kinorhyncha Reinhard, 1885',
 'Rotifera',
 'Phoronida',
 'Mollusca',
 'Ctenophora Eschscholtz, 1829',
 'Brachiopoda Duméril, 1805',
 'Hemichordata Bateson, 1885',
 'Stomochordata',
 'Onychophora Grube, 1853',
 'Tardigrada Doyère, 1840',
 'Sipuncula',
 'Sipunculoidea Sedgwick, 1898',
 'Acanthocephala Rudolphi, 1802',
 'Xenacoelomorpha Philippe, Brinkmann, Copley, Moroz, Nakano, Poustka, Wallberg, Peterson & Telford, 2011',
 'Bryozoa',
 'Nematoda',
 'Nemertea',
 'Priapulida Théel, 1906',
 'Porifera Grant, 1836']

In [41]:
taxon.filter_by_phylum(found_phyla)
taxon.pandas

Filtering Phylum: 100%|██████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:15<00:00,  6.34it/s, Exact match found=37, Synonyms found=7, Parents found=5, Total filtered=108094]


,taxonID,parentNameUsageID,acceptedNameUsageID,originalNameUsageID,scientificNameID,datasetID,taxonomicStatus,taxonRank,scientificName,scientificNameAuthorship,...,nomenclaturalCode,nomenclaturalStatus,taxonRemarks,references,kingdom,phylum,class,order,family,genus
0,5T6MX,,5T6MX,,0,,accepted,unranked,Biota,,...,,nomen legitimum,,,,,,,,
2,43HK,624LX,43HK,,845ab74c-f98f-4bbd-9172-bdc3ca854cd7,1171,accepted,genus,Derophthalma,,...,,,,,Animalia,Arthropoda,Insecta,Hemiptera,Miridae,Derophthalma
3,63GPQ,624LX,63GPQ,,ecaf98e7-5e77-4ec5-b96f-9a158477f23d,1171,accepted,genus,Osornocoris,,...,,,,,Animalia,Arthropoda,Insecta,Hemiptera,Miridae,Osornocoris
4,6VKD,624LX,6VKD,,c71d9a67-3632-4011-b4ba-66642ae88516,1171,accepted,genus,Prepops,,...,,,,,Animalia,Arthropoda,Insecta,Hemiptera,Miridae,Prepops
5,CHN,N,CHN,,a87d1f7f-8fd6-4474-ae29-423d135293c9,,accepted,phylum,Echinodermata,,...,,nomen legitimum,,,Animalia,Echinodermata,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212343,6SGR8,6BNJ,4BPSG,,74f310f0-9fb2-489a-8df9-32649a737c2a,1037,synonym,species,"Odynerus simplicicornis de Saussure, 1854","de Saussure, 1854",...,ICZN,,,,Animalia,Arthropoda,Insecta,Hymenoptera,Eumenidae,Pachodynerus
212344,5FP5F,F3MS,5FP5F,,6f3df4b9-7951-4e60-9a5f-969c390d2e32,1037,accepted,subspecies,Antodynerus sheffieldi bellatuloides Giordani ...,"Giordani Soika, 1990",...,ICZN,,,http://www.zobodat.at,Animalia,Arthropoda,Insecta,Hymenoptera,Eumenidae,Antodynerus
212345,36TKQ,6XPD,4N56P,,ba2b889c-62d5-4a16-bc41-3abfcee8ce2d,1037,synonym,species,"Discoelius chilensis de Saussure, 1852","de Saussure, 1852",...,ICZN,,,,Animalia,Arthropoda,Insecta,Hymenoptera,Eumenidae,Protodiscoelius
212346,36TLC,6XPD,4N56Q,,c45917ae-30d4-4c2d-a9e7-ae02d581375d,1037,synonym,species,"Discoelius valdiviensis Giordani Soika, 1958","Giordani Soika, 1958",...,ICZN,,,,Animalia,Arthropoda,Insecta,Hymenoptera,Eumenidae,Protodiscoelius


In [42]:
print(chilean_col)
for extension in chilean_col.extensions:
    print(extension)

col-clb-9910 [Core: http://rs.tdwg.org/dwc/terms/Taxon, Entries: 108094]
Extension:
	class: http://rs.gbif.org/terms/1.0/Distribution
	filename: Distribution.tsv
	content: 29128 entries
Extension:
	class: http://rs.gbif.org/terms/1.0/SpeciesProfile
	filename: SpeciesProfile.tsv
	content: 27559 entries
Extension:
	class: http://rs.gbif.org/terms/1.0/VernacularName
	filename: VernacularName.tsv
	content: 32433 entries


In [43]:
chilean_col.to_file("data/chilean_col_var.zip")

Writing data http://rs.tdwg.org/dwc/terms/Taxon: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 108094/108094 [00:08<00:00, 13494.84line/s]
Writing data http://rs.gbif.org/terms/1.0/Distribution: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29128/29128 [00:00<00:00, 67895.26line/s]
Writing data http://rs.gbif.org/terms/1.0/SpeciesProfile: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 27559/27559 [00:00<00:00, 88846.44line/s]
Writing data http://rs.gbif.org/terms/1.0/VernacularName: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32433/32433 [00:00<00:00, 140962.62line/s]


<table>
    <tr>
        <td colspan="3" style="text-align: center;"><p>BIODATA - <a href="https://ieb-chile.cl/en/" target="_blank">Institute of Ecology and Biodiversity</a> © 2024</p></td>
    </tr>
</table>